# How to generate data files for tabulated rates

These tabulated reaction rates are included along with library files
describing how the tables should be read.

Tabulated reactions of the form `A -> B` are supported, where the
rates are tabulated in a two-dimensional space of:

- the product of density and electron fraction
- temperature

This directory includes tabulated rates obtained from Suzuki et al.,
2016, ApJ 817:163, downloaded from
http://w3p.phys.chs.nihon-u.ac.jp/~suzuki/data2/link.html

The suzuki data table are in the follower form:
```
!23Ne -> 23F,  e-capture  with screening effects
!USDB           Q=-8.4635 MeV
!Transitions from 5/2+, 1/2+, 7/2+, 3/2+ states of 23Ne are included.
!Experimental data are used.
!
!Log(rhoY) Log(T) mu    dQ        Vs        e-cap-rate    nu-energy-loss  gamma-energy
!                 (MeV)    (MeV)     (MeV)        (1/s)          (MeV/s)        (MeV/s)

   7.00  7.00   1.2282   0.0279   0.0077  -0.50000E+03  -0.50000E+03  -0.50000E+03
   7.00  7.20   1.2270   0.0278   0.0077  -0.50000E+03  -0.50000E+03  -0.50000E+03
   7.00  7.40   1.2253   0.0275   0.0077  -0.50000E+03  -0.50000E+03  -0.50000E+03
```
First, we read it.

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

f = open("/Users/sailor/Desktop/A23_Ne_F.dat","r")
data = f.readlines()  # data is a list. each element is a line of "A23_Ne_F.dat"
f.close()

Next, we find the header line of the data file

In [2]:
inde = []  
for i in range(len(data)):
    if data[i][0] == '!':     # if the line start as "!" , this is a header line
        inde.append(i)     # store the detected index into this list

header = data[inde[0]:len(inde)]
header

['!23Ne -> 23F,  e-capture  with screening effects\n',
 '!USDB           Q=-8.4635 MeV\n',
 '!Transitions from 5/2+, 1/2+, 7/2+, 3/2+ states of 23Ne are included.  \n',
 '!Experimental data are used.\n',
 '!\n',
 '!Log(rhoY) Log(T) mu    dQ        Vs        e-cap-rate    nu-energy-loss  gamma-energy\n',
 '!                 (MeV)    (MeV)     (MeV)        (1/s)          (MeV/s)        (MeV/s)  \n']

We modify the last 2 lines of header to meet the requirments of pynucastro.

In [3]:
header[-2] = '!rhoY        T            mu           dQ           Vs           e-cap-rate   nu-energy-loss gamma-energy\n'
header[-1] = '!g/cm^3      K            erg          erg          erg          1/s          erg/s        erg/s\n'
header

['!23Ne -> 23F,  e-capture  with screening effects\n',
 '!USDB           Q=-8.4635 MeV\n',
 '!Transitions from 5/2+, 1/2+, 7/2+, 3/2+ states of 23Ne are included.  \n',
 '!Experimental data are used.\n',
 '!\n',
 '!rhoY        T            mu           dQ           Vs           e-cap-rate   nu-energy-loss gamma-energy\n',
 '!g/cm^3      K            erg          erg          erg          1/s          erg/s        erg/s\n']

Then we convert the data table from a 1D string array to a 2D float array.

In [4]:
del data[inde[0]:len(inde)]  # delete the header lines

# change the list ["1.23 3.45 5.67\n"] into the list ["1.23","3.45","5.67"]
data1 = []
for i in range(len(data)):
    data1.append(re.split(r"[ ]",data[i].strip('\n')))
    
# delete all the "" in each element of data1
for i in range(len(data1)):
    while '' in data1[i]:
        data1[i].remove('')
        
# delete all [] in data1
while [] in data1:
    data1.remove([])

# convert the type from string into float    
data2 = []
for i in range(len(data1)):
    data2.append([])
    for j in range(len(data1[i])):
        data2[i].append(float(data1[i][j]))
data2[0:3]

[[7.0, 7.0, 1.2282, 0.0279, 0.0077, -500.0, -500.0, -500.0],
 [7.0, 7.2, 1.227, 0.0278, 0.0077, -500.0, -500.0, -500.0],
 [7.0, 7.4, 1.2253, 0.0275, 0.0077, -500.0, -500.0, -500.0]]

We need to convert the unit to meet the requirements of pynucastro
- convert log(rhoY/(g/cm^3)) to rhoY/(g/cm^3) 
- convert log(T/K) to T/K
- convert Mev to erg in mu
- convert Mev to erg in dQ
- convert Mev to erg  in Vs
- convert log(rate/(s^-1)) to rate/(s^-1) in e-cap-beta-rate
- convert log(rate/(Mev/s)) to rate/(erg/s) in nu-energy-loss
- convert log(rate/(Mev/s)) to rate/(erg/s) in gamma-energy

In [5]:
for i in range(len(data2)):
        data2[i][0] = "%e"%(np.power(10,data2[i][0]))
        data2[i][1] = "%e"%(np.power(10,data2[i][1]))
        data2[i][2] = "%e"%(data2[i][2]*1.60218e-6)
        data2[i][3] = "%e"%(data2[i][3]*1.60218e-6)
        data2[i][4] = "%e"%(data2[i][4]*1.60218e-6)
        data2[i][5] = "%e"%(np.power(10,data2[i][5])) 
        data2[i][6] = "%e"%(np.power(10,data2[i][6])*1.60218e-6)
        data2[i][7] = "%e"%(np.power(10,data2[i][7])*1.60218e-6)
data2[0]

['1.000000e+07',
 '1.000000e+07',
 '1.967797e-06',
 '4.470082e-08',
 '1.233679e-08',
 '0.000000e+00',
 '0.000000e+00',
 '0.000000e+00']

Then we write the new data into a data file that pynucastro can read.

In [6]:
# convert the type from float to string
data3 = []
for i in range(len(data2)):
    data3.append([])
    for j in range(len(data2[i])):
        data3[i].append(str(data2[i][j]))

f = open("/Users/sailor/Desktop/23Ne-23F_electroncapture.dat", "w") 
for i in range(len(header)):
    f.write(header[i])
for i in range(len(data3)):
    for j in range(len(data3[i])-1):
        f.write(data3[i][j])
        f.write(' ')
    f.write(data3[i][len(data3[i])-1])
    f.write('\n')
f.close()

Next we need to generate a rate file to describe how to read the table file. (You can go to http://pynucastro.github.io/pynucastro/networks.html#tabular-rates to read details)

In [7]:
rate_file = open("/Users/sailor/Desktop/ne23--f23-toki","w") 
rate_file.write("t\n       "+"ne23"+" "+"f23"+"\n"+"23Ne-23F_electroncapture.dat"+"\n152\n39") 
rate_file.close()

In the end, we need to copy the two generated files ```23Ne-23F_electroncapture.dat``` and ```ne23--f23-toki``` into the folder ```/pynucastro/library/tabular/```. Then pynucastro can use them.